## 1. explain note book
* chose only patients that have Age greater or equal 17.9
* 1) louad the means and std from the file that we constrain in the train phase
* 2) based on the literature and some statistic calcules (ex: missig rate), we constrain the felowing sub sets of columns:
    * 2.1) columns with time series data and < 15% missing data for exemple
    * 2.2) columns with continuous data to be beened and >= 15% missing data for exemple
* 3) drop initul columns
* 4) replace NaN values of the continious variable with forward/backward methods
    * remarque: the remaining NaN values of the continious variable will be skiped by a sequence in the training phase
    * standardiside the not binary continious variable
* 5) replace NaN values of the binary variable with forward/backward methods
    * remarque: replace the remaining NaN values of the binary variable with 1.0 (if they are all NaN, we take "Gender as 1.0 (hman) and Uni1=1.0 (MICU) " )

* remarque: we will process the other variables to be beened in the next steps (not heer), when we will constrain the sequenses
    * ie: we will stadardisided it in the (6.3.1) step,
    * the remaing NaN values will be replaced in the (6.3.2) step by np.pi(3.14) to be skiped in the infering model step.
* 6) windowed data in 16 as window feature length.
    * 6.1) extended data to length window, data length is miner than window length.

    * 6.1) get all the continuous variables into one group for a window data
    * 6.2) skiped the window in that, any of the continuous variables is NaN
    * 6.3) process each of the variables to be binned
        * 6.3.1) standardised  the not binary variables to be binned
        * 6.3.2) replace NaN with pi the ramining  data (laboratire test) to will be masking in the infering model phase
    
    * 6.4) generate a new window data, when we d'ont have data as input, car in the step 4, it possible to generate a null data.
    * 6.5) make the prediction probabilities for each window data
    * 6.7) make the prediction labels for each window data based on the obtained threshold that maximize the accuracy metric
* 7) take the mean of the prediction probabilities as probability output of this patient data at time t
* 8) take the most frequent labels as label of this patient data at time t
* 9) save all prediction probability and all prediction label by each patient in a directory output
    * 9.1) ie: if we have 100 paient, we obtain 100 files contain thair probability and label prediction.
* 10) performing our results by calculate the auroc, auprc, accuracy, f_measure and normalized_observed_utility metrics.

# 2. libreries  import

In [1]:
pip install fidle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.7 MB/s eta 0:00:00


In [2]:
import numpy as np, os, sys
import pandas as pd
import tensorflow as tf
import shutil
import pickle
from tensorflow.keras.utils import to_categorical # to transform data into categrial type
import fidle
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 200)

# 3. parametres

In [3]:
ROOT_DIR = os.path.dirname(os.path.abspath('vedio RNN'))
#----------------------------------------------------inputs---------------------------------
# Setup paths
#DATA_DIR = ROOT_DIR + '/data/test/1000AB'
#DATA_DIR = ROOT_DIR + '/data/test/4AB'
#DATA_DIR = ROOT_DIR + '/data/test/500A'
#DATA_DIR = ROOT_DIR + '/data/test/500B'
#DATA_DIR = ROOT_DIR + '/data/test/40336AB'

#DATA_DIR = ROOT_DIR + '/data/test/20AB'
#DATA_DIR = ROOT_DIR +'/drive/MyDrive/20AB'

#DATA_DIR = ROOT_DIR + '/data/test/20336A'
DATA_DIR = ROOT_DIR +'/drive/MyDrive/20336A'

#DATA_DIR = ROOT_DIR + '/data/test/20000B'
#DATA_DIR = ROOT_DIR +'/drive/MyDrive/20000B'
#DATA_DIR = ROOT_DIR + '/data/test/p100005 B'
#DATA_DIR = ROOT_DIR + '/data/test/p000009 A'

#---------------------------
dt_early   = -12
dt_optimal = -6
dt_late    = 3

#------------------------
#for data in ['all_df', 'train_set', 'test_set','val_set']:
    #df=load_pickle( DATA_DIR + '/from_raw/XY/'+ data + '_x_y_raw.pickle')
    # loud the ids of those who eventually develop sepsis fromp labels before/after processed data.
    #raw_labels=load_pickle(DATA_DIR + '/from_raw/Y/'+ data + '_original_labels.pickle') # raw eventual sepsi avec index id , time
    #---------------
    #processed_labels=load_pickle( DATA_DIR + '/processed/Y/'+ data + '_processed_labels.pickle') # processed eventual sepsi avec index patient , ICULOS
    #---------------
    # convert a time series to columns, that can be used in decision tree or random forest
    #data_cont_convert=pd.DataFrame(data_cont.reshape(data_cont.shape[0],data_cont.shape[1]*data_cont.shape[2]),columns=[f'{col}'+'_hour'+f'{i+1}' for  i in (range(16)) for col in ['HR', 'MAP', 'O2Sat', 'SBP', 'Resp']])
#----------------

#scale       =1 #pourcentage of dataset to be used (1=all dataset)
# if using future/past split method
#train_prop  =.8 #poucentage for train (the rest being for the test)
# if using paire/impaire split method
#train_prop  =80 #poucentage for train (the rest being for the test)

#---------------------------------results--------------------------------------------
#for data in ['all_df', 'train_set', 'test_set','val_set']:
    # Save data values x
    #save_pickle(windowed_df, DATA_DIR + '/processed/X/'+ data +'_processed.pickle')
    # Save data values x data_cat
    #save_pickle(data_cat, DATA_DIR + '/processed/X/data_cat/'+ data +'_data_cat_processed.pickle')
    # Save data values x data_cont
    #save_pickle(data_cont, DATA_DIR + '/processed/X/data_cat/'+ data +'_data_cont_processed.pickle')
    #save data labels y
    #save_pickle(labels, DATA_DIR + '/processed/Y/'+ data + '_processed_labels.pickle')
    #save data  x_y
    #save_pickle(windowed_data_df, DATA_DIR + '/processed/XY/'+ data + '_x_y_processed.pickle')
    #save categorial labels
    #save_pickle(categorial_labels, DATA_DIR + '/categorial/'+ data + '_categorial_processed_labels.pickle')
    # Save the ids of those who eventually develop sepsis fromp labels before/after processed data.
    #save_pickle(ids, DATA_DIR + '/raw eventual sepsis/'+ data + '_raw_ids_eventual.pickle')
    #---------------
    #save_pickle(ids, DATA_DIR + '/processed eventual sepsis/'+ data + '_processed_ids_eventual.pickle')
    #---------------
    # convert a time series to columns, that can be used in decision tree or random forest
    #save_pickle(data_cont_convert, DATA_DIR + '/processed tree/processed/'+ data +'decision tree.pickle')

# 4. functions

In [4]:
def load_challenge_data(file):
    with open(file, 'r') as f:
        header = f.readline().strip()
        column_names = header.split('|')
    return data

def save_pickle(obj, filename, use_dill=False, protocol=4):
    """ Basic pickle/dill dumping """
    create_folder_if_not_exist(filename)
    with open(filename, 'wb') as file:
        if not use_dill:
            pickle.dump(obj, file, protocol=protocol)
        else:
            dill.dump(obj, file)

def create_folder_if_not_exist(filename):
    #Makes a folder if the path does not already exist
    os.makedirs(os.path.dirname(filename), exist_ok=True)


def load_pickle(filename, use_dill=False):
    """ Basic pickle/dill loading function """
    with open(filename, 'rb') as file:
        if not use_dill:
            obj = pickle.load(file)
        else:
            obj = dill.load(file)
    return obj

#df=load_pickle(DATA_DIR + '/processed/XY/test_set_x_y_processed.pickle')
def test_nan(df,sam=False):

    dataset_nan=df.isna().sum().tolist()
    dataset_col=df.columns.tolist()
    #datset_desc=[code2desc[c] for c in dataset_col]
    # show all of that
    #fidle.utils.subtitle('our dataset columns?')
    df_desc=pd.DataFrame({'codes': dataset_col, 'NAN':dataset_nan})
    df_desc.sort_values(by=['NAN'],inplace=True)
    if(sam):
      print(" sum nan is :", np.sum(df_desc['NAN']))
    else:

      fidle.utils.subtitle('our inhanced dataset ?')
      display(df_desc.style.set_properties(**{'text-align': 'left'}))
    return None
#--------------------------------------
def frequent_values_from_larger_consecutif_label(l,return_greater_list=False):
    glob=[]
    consec=[]
    if(np.sum(l)==0):
        val_label_frequent=0.0
        greater_list=l
    else:
        if(np.sum(l)==len(l)):

            val_label_frequent=1.0
            greater_list=l
        else:

            k=0
            i=0
            while (i<len(l)-1):
                k=i
                consec=[]
                consec.append(l[k])
                while (k<len(l)-1 and  l[k]==l[k+1]):
                    consec.append(l[k+1])
                    k=k+1
                    i=i+1
                i=i+1
                glob.append(consec)
            greater_list=[]
            if (len(glob)>1):

                max=np.max([len(glob[i]) for i in range(len(glob))])
                for i in range(len(glob)):
                    if (len(glob[i])==max):
                        ss=glob[i]
                        greater_list.append(ss)
                val_label_frequent=ss[0]
            else:
                val_label_frequent=l[0]
                greater_list=l
    if(return_greater_list==True):
        return val_label_frequent, greater_list
    else:
        return val_label_frequent

def sample_great_than_miss_value(x,i):
    a=[]

    for i in range(0,x.shape[0]):
        if (x.iloc[i,0:5].isna().sum(axis=0)<i):
            a.append( x.iloc[i,0:x.shape[1]])

    h=pd.DataFrame(a,columns=x.columns)
    #display('dddddddddddd   ',h.head())
    return h

def DropFeatures(features, df):
    # Reduce to selected features
    if features is not None:
        df = df.drop(features, axis=1)
    return df
#-----------------------------------------------------
def data_processing_and_transformation(data,df_mean_std,normalise=False, win_len=16, dt_optimal = -6, pred_len=1):

  #---------------------------------------------------------------
  """cols_cont = ['HR', 'MAP','SBP', 'O2Sat', 'Temp', 'Resp']

  cols_to_bin = ['Glucose', 'Potassium', 'Hct', 'FiO2', 'Hgb', 'pH',
                'BUN', 'WBC', 'Magnesium', 'Creatinine', 'Platelets', 'Calcium', 'PaCO2',
                'BaseExcess', 'Chloride', 'HCO3', 'Phosphate', 'PTT', 'Lactate', 'AST',
                'Alkalinephos',  'TroponinI', 'Fibrinogen', 'Bilirubin_direct']"""

  droped_cols=['SaO2','Bilirubin_total','DBP','EtCO2','HospAdmTime']

  related_col__Bilirubin=['Bilirubin_direct','Bilirubin_total']

  related_col_BP=['DBP','MAP','SBP']

  index_columns=['ICULOS', 'patient','patient1']
  index_raw_df=['time','id']


  cols_cont=[ 'HR', 'MAP','SBP',  'DBP', 'O2Sat' , 'Resp', 'Temp','Age']

  cols_to_bin=[ 'Glucose','Potassium','Hct','FiO2', 'Hgb', 'pH',
                    'BUN', 'WBC','Magnesium', 'Creatinine','Platelets', 'Calcium','PaCO2',
                    'BaseExcess','Chloride', 'HCO3','Phosphate','PTT','Lactate', 'AST',
                    'Alkalinephos','TroponinI',  'Fibrinogen','Bilirubin_direct',
                    'SaO2','Bilirubin_total'
                    ]



  binary_column= ['Unit1', 'Gender']
  raw_taget_column=['SepsisLabel']
  #---------------------------------------------------------------
  """  # based on the literature and some statistic calcules (ex: missig rate), we constrain the feloxing sub sets of columns:
  #columns with time series data and < 15% missing data for exemple

  cols_cont = ['HR', 'MAP', 'O2Sat', 'SBP', 'Resp']

  #columns with continuous data to be beened and >= 15% missing data for exemple
  cols_to_bin = ['Unit1', 'Gender',  'Age', 'DBP', 'Temp', 'Glucose', 'Potassium', 'Hct',
                  'FiO2', 'Hgb', 'pH', 'BUN', 'WBC', 'Magnesium', 'Creatinine', 'Platelets',
                  'Calcium', 'PaCO2',
                  'BaseExcess', 'Chloride', 'HCO3', 'Phosphate', 'SaO2', 'PTT', 'Lactate', 'AST',
                  'Alkalinephos', 'Bilirubin_total', 'TroponinI', 'Fibrinogen', 'Bilirubin_direct']

  column_inf=['ICULOS','patient','patient1']
  binary_cols = ['Unit1', 'Gender']
  target='SepsisLabel'"""
  #---------------------------------------------------------------

  df=pd.concat([data[cols_cont],data[cols_to_bin],data[binary_column],data[index_columns],data[raw_taget_column]],axis=1)

  df=sample_great_than_miss_value(df,4)
  #df=df[(df[cols_cont].isnull().sum(axis=1))<=5]
  #display(' df display   ', df.head())
  is_sepsis=False
  if np.any(df['SepsisLabel']):
      is_sepsis=True
      t_sepsis = np.argmax(df['SepsisLabel']) -(dt_optimal)

  # replace NaN values of the continious variable with forward/backward methods
  df[cols_cont] = (df[cols_cont].fillna(method='bfill').fillna(method='ffill'))

  # remarque: the remaining NaN values of the continious variable will be skiped by a sequence in the training phase
  # standardiside the not binary continious variable

  #Standard Scalaer and make transoforme colmnns
  if(normalise):
    for col in cols_cont:
        if (df_mean_std[col]['std']!=0):
          df[col]=np.nan_to_num(df[col]-np.nan_to_num(df_mean_std[col]['mean']))/np.nan_to_num(df_mean_std[col]['std'])
        else:
          df[col]=1
  # replace NaN values of the binary variable with forward/backward methods
  df[binary_column] = (df[binary_column].fillna(method='bfill').fillna(method='ffill'))

  # replace the remaining NaN values of the binary variable with 1.0 (if they are all NaN, we take "Gender as 1.0 (hman) and Uni1=1.0 (MICU) " )
  df[binary_column] = (df[binary_column].fillna(1.0))
  # remarque: we treat NaN values of the other variables in the next step (not hear), when we constrain the sequense ie: we will replace it by median
  # than, we stadardisided
  # the remaing NaN values will replaced by np.pi(3.14) to be skiped in the infering model step.
  #display(df.head(50))

  windowed_data_list = []
  #N =len(df): if  nonsepsis patient #N=t_sepsis if sepsis patient
  N = len(df)
  if (is_sepsis==True):
      N=t_sepsis
  #win_len = 16
  i = 0
  # ------------------- augmente the size of df by 0 when we have louer size data مازال
  # nous avon résoudre ce problem, on utilisant que les patient qui ont length patient> win_len+len_pred
  if (N<win_len):
      if(len(df!=0)):
          df2=pd.DataFrame(np.zeros((win_len-N+1,df.shape[1])),columns=df.columns)
          #df2='unknoun'
          #df = pd.concat([df,df2],axis=0).reset_index(drop=True)
          df = pd.concat([df,df2])#sort_index(ascending=True)
          N = len(df)
      else:
        df=pd.DataFrame(np.zeros((win_len-N+1,df.shape[1])),columns=df.columns)
        N = len(df)

  """input_data = data[:-10]
        targets = data[10:]
        dataset = tf.keras.utils.timeseries_dataset_from_array(
            input_data, targets, sequence_length=10)
        for batch in dataset:
          inputs, targets = batch
          assert np.array_equal(inputs[0], data[:10])  # First sequence: steps [0-9]
          # Corresponding target: step 10
          assert np.array_equal(targets[0], data[10])
          break"""
  while(i+win_len+pred_len <= N):
      tmp_data = df.iloc[i:i+win_len]
      tmp_label = df.iloc[i+win_len:i+win_len+pred_len]
      tmp_label=tmp_label['SepsisLabel']
      tmp_label=tmp_label.iloc[:]

      if np.isnan(tmp_label).any():
          tmp_patient =tmp_patient1='unknoun'
          tmp_label=int(0)
      else:
          #tmp_label = int(frequent_values_from_larger_consecutif_label(list(tmp_label)))
          tmp_patient =tmp_patient1= str(tmp_data['patient'].values[0])
          tmp_label = int(tmp_data['SepsisLabel'].values[0])
      #slide the window forward
      i = i+1
      #get all the continuous variables into one group
      X_cont = tmp_data[cols_cont]
      """X_cont = X_cont.fillna(method='bfill')
      if((X_cont[cols_cont].isnull().sum(axis=1))<=4 or tmp_patient=='unknoun'):
        X_cont = X_cont.fillna(method='ffill')
      #replace NaN with (-pi) the ramining  data cont (continues) to will be masking in the infering model phase
      #X_train_cat=win.loc[:, cols_cont]
      X_cont=X_cont.fillna(-(np.pi))"""


      #if (np.isnan(X_cont).sum(axis=1)>2).any(): continue
      X_cont = X_cont.values

      # skiped the window in that any of the continuous variables is nan/NaN
      if (np.isnan(X_cont).any() or tmp_patient=='unknoun'): continue #*********** or aplay other model of prediction (forcasting)

      #process each of the variables to be binned (represent its one by the median in this window)
      X_binned_dict = {}
      for col_to_bin in cols_to_bin:
          tmp_val = tmp_data[col_to_bin].median()

          # standardised  the not binary variables to be binned
          if(normalise):

            if col_to_bin not in ['Gender', 'Unit1', 'ICULOS','patient','patient1']:
              #if  tmp_val!=np.nan:
              if (df_mean_std[col_to_bin]['std']!=0):
                #and df_mean_std[col_to_bin]['std']!=np.nan and df_mean_std[col_to_bin]['mean']!=np.nan):
                tmp_val =np.nan_to_num (tmp_val-np.nan_to_num(df_mean_std[col_to_bin]['mean']))/np.nan_to_num(df_mean_std[col_to_bin]['std'])
              else:
                tmp_val=1

          X_binned_dict[col_to_bin] = tmp_val

      #package it all into a dictionary
      tmp_dict = X_binned_dict
      tmp_dict['X_cont'] = X_cont
      tmp_dict['ICULOS']=(tmp_data['ICULOS'].median()  )
      #tmp_dict['ICULOS']=(tmp_data['ICULOS'].values[0])
      tmp_dict['Gender']=(tmp_data['Gender'].values[0])
      tmp_dict['Unit1']=(tmp_data['Unit1'].values[0])
      tmp_dict['patient'] = tmp_patient
      tmp_dict['patient1'] = tmp_patient1
      tmp_dict['label'] = tmp_label
      windowed_data_list.append(tmp_dict)

  win = pd.DataFrame(windowed_data_list,columns=cols_to_bin+['X_cont']+['ICULOS']+['Gender']+['Unit1']+['patient']+['patient1']+['label'])
  # if setting patient and ICULOS as index data
  win = win.reset_index().set_index(['patient', 'ICULOS']).sort_index(ascending=True)

  #replace NaN with pi the ramining  data categ (laboratire) to will be masking in the infering model phase
  X_train_cat=win.loc[:, cols_to_bin]
  X_train_cat=X_train_cat.fillna(np.pi)



  X_train_cont=win.loc[:, 'X_cont']

  win_data=win.loc[:, ['Gender','Unit1','patient1','label']]

  windowed_data_df =pd.concat([X_train_cat,X_train_cont,win_data],axis=1).sort_index(ascending=True)

  #windowed_df = pd.concat([X_train_cat,X_train_cont],axis=1).sort_index(ascending=True)
  #print('windowed_data_df.shape[0]   ',windowed_data_df.shape[0])
  if(windowed_data_df.shape[0]>0):

      data_cat = pd.DataFrame(windowed_data_df,columns=cols_to_bin)

      data_cont = np.asarray(list(windowed_data_df['X_cont'].values))
      """if (len(data_cont.shape) == 1):
        data_cont=    np.reshape(data_cont,(data_cont.shape[0],1,1))"""
        #data_cont.shape[1]=win_len
        #data_cont.shape[2]=len(cols_cont)


      #data_cont=    np.reshape((data_cont.shape[0],win_len,len(cols_cont)))
      #print ('data_cont.shape   ', data_cont.shape)
      #windowed_df = pd.concat([X_train_cat,X_train_cont],axis=1).reset_index(drop=True)
      #windowed_df = pd.concat([X_train_cat,X_train_cont],axis=1).sort_index(ascending=True)
      #labels=windowed_data_df['label']

      #-----------------------format of data to be used in other model such as decision tree-----
      data_cont_convert=pd.DataFrame(data_cont.reshape(data_cont.shape[0],data_cont.shape[1]*data_cont.shape[2]),columns=[f'{col}'+'_H'+f'{i+1}' for  i in (range(win_len)) for col in cols_cont])
      windowed_data_df_tree =pd.concat([data_cont_convert,data_cat.reset_index(drop=True),win_data.reset_index(drop=True)],axis=1).sort_index(ascending=True)
      #---------------------------

  else:
      data_cat = pd.DataFrame(np.zeros((1,windowed_data_df[cols_to_bin].shape[1])),columns=cols_to_bin)
      data_cont=    np.zeros((1,win_len,len(cols_cont)))
      #print('level None')
      #----------------------------
      data_cont_convert=pd.DataFrame(data_cont.reshape(data_cont.shape[0],data_cont.shape[1]*data_cont.shape[2]),columns=[f'{col}'+'_H'+f'{i+1}' for  i in (range(win_len)) for col in cols_cont])
      windowed_data_df_tree =pd.concat([data_cont_convert,data_cat.reset_index(drop=True),win_data.reset_index(drop=True)],axis=1).sort_index(ascending=True)
      #---------------------------

  return  windowed_data_df, windowed_data_df_tree

# 5. main call

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Model LSTM and Other

In [6]:
# louad the means and std from the file that we constrain in the train phase:
df_mean_std=load_pickle(DATA_DIR + '/mean/mean_std.pickle')
ss=np.array(df_mean_std)
hh=np.nan_to_num(ss)
df_mean_st=pd.DataFrame(hh,index=['mean','std'], columns=df_mean_std.columns)
win_len=16
pred_len=1
dt_optimal = -6
normalise=True
for data in ['all_df', 'train_set', 'test_set', 'sub_train_set', 'val_set']:

  #for data in ['test_set', 'sub_train_set', 'val_set']:#, 'test_set']:

  windowed_data_df_list, windowed_data_df_tree_list=[],[]
  #data_cat_list,data_cont_list, windowed_df_list,labels_list, windowed_data_df_list, windowed_data_df_tree_list=[],[],[],[],[],[]
  df=load_pickle( DATA_DIR + '/from_raw/XY/'+ data + '_x_y_raw.pickle')
  # step of prprocessing global such as age>=18, etc...
  df=df[df['Age']>=17.9]
  #-----------
  grouped_by_patient = df.groupby('patient')
  for patient, group in grouped_by_patient:
    N=len(group)
    is_sepsis=False
    if np.any(group['SepsisLabel']):
        is_sepsis=True
        t_sepsis = np.argmax(df['SepsisLabel']) -(dt_optimal)
        N=t_sepsis
    if(N>win_len+pred_len):
      windowed_data_df, _= data_processing_and_transformation(group,df_mean_st,normalise,win_len, dt_optimal , pred_len)
      windowed_data_df_list.append(windowed_data_df)
      #windowed_data_df_tree_list.append(windowed_data_df_tree)
  windowed_data_df=pd.concat(windowed_data_df_list)

  # droped None level
  windowed_data_df=windowed_data_df[windowed_data_df['patient1']!=0]
  #windowed_data_df_tree=pd.concat(windowed_data_df_tree_list)
  #save data  x_y
  save_pickle(windowed_data_df, DATA_DIR + '/processed1/XY/'+ data + '_x_y_processed.pickle')

  #save_pickle(windowed_data_df_tree, DATA_DIR + '/processed/XY/'+ data + '_decision_tree_x_y_processed.pickle')

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
/usr/local/lib/python3.10/dist-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.10/dist-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.10/dist-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.10/dist-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.10/dist-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.10/dist-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of e

KeyboardInterrupt: ignored

Model Decision tree or Other

In [ ]:
# louad the means and std from the file that we constrain in the train phase:
"""df_mean_std=load_pickle(DATA_DIR + '/mean/mean_std.pickle')
ss=np.array(df_mean_std)
hh=np.nan_to_num(ss)
df_mean_st=pd.DataFrame(hh,index=['mean','std'], columns=df_mean_std.columns)"""
df_mean_st=1
win_len=16
pred_len=1
dt_optimal = -6
normalise=False
for data in ['all_df', 'train_set', 'test_set', 'sub_train_set', 'val_set']:

  #for data in ['test_set', 'sub_train_set', 'val_set']:#, 'test_set']:

  windowed_data_df_tree_list=[]
  #data_cat_list,data_cont_list, windowed_df_list,labels_list, windowed_data_df_list, windowed_data_df_tree_list=[],[],[],[],[],[]
  df=load_pickle( DATA_DIR + '/from_raw/XY/'+ data + '_x_y_raw.pickle')
  # step of prprocessing global such as age>=18, etc...
  df=df[df['Age']>=17.9]
  #-----------
  grouped_by_patient = df.groupby('patient')
  for patient, group in grouped_by_patient:
    N=len(group)
    is_sepsis=False
    if np.any(group['SepsisLabel']):
        is_sepsis=True
        t_sepsis = np.argmax(df['SepsisLabel']) -(dt_optimal)
        N=t_sepsis
    if(N>win_len+pred_len):
      _, windowed_data_df_tree= data_processing_and_transformation(group,df_mean_st,normalise,win_len, dt_optimal , pred_len)
      #windowed_data_df_list.append(windowed_data_df)
      #if windowed_data_df_tree != None:

      windowed_data_df_tree_list.append(windowed_data_df_tree)
  #windowed_data_df=pd.concat(windowed_data_df_list)
  windowed_data_df_tree=pd.concat(windowed_data_df_tree_list)

  # droped None level
  windowed_data_df_tree=windowed_data_df_tree[windowed_data_df_tree['patient1']!=0]

  #save data  x_y
  #save_pickle(windowed_data_df, DATA_DIR + '/processed/XY/'+ data + '_x_y_processed.pickle')

  save_pickle(windowed_data_df_tree, DATA_DIR + '/processed1//tree/XY/'+ data + '_decision_tree_x_y_processed.pickle')

# 6. few statistics

statistic LSTM and Other

In [ ]:
for data,df_value in [('all_df','all df_values'), ('train_set','train_values'),
                            ('test_set','test_values'),
                           ('sub_train_set','sub_train_values'), ('val_set','val_values') ]:

    fidle.utils.subtitle('-------------' + f'{data}'+'   ----------------')
    x_y_values=load_pickle(DATA_DIR + '/processed1/XY/'+ data + '_x_y_processed.pickle')
    #x_y_values=load_pickle(DATA_DIR + '/processed/XY/'+ data + '_decision_tree_x_y_processed.pickle')

    #display(f'{df_value}'+' display  x y ',x_y_values.head())

    #----------------------
    if(data=='all_df'):
        all_size=x_y_values.shape[0]
        all_patient=len(x_y_values['patient1'].unique())
        print('pourcentage **** all df*************   ', '100' +'  %')
        print('pourcentage **** all patient*************   ', '100' +'  %')
        print("\n")
        train_x_y_values=load_pickle( DATA_DIR + '/processed1/XY/train_set_x_y_processed.pickle')

        train_size=train_x_y_values.shape[0]
        train_patient=len(train_x_y_values['patient1'].unique())
        print('pourcentage **** train_set*************   ',train_x_y_values.shape[0]* 100/all_size )

        test_x_y_values=load_pickle( DATA_DIR + '/processed1/XY/test_set_x_y_processed.pickle')

        test_size=test_x_y_values.shape[0]
        test_patient=len(test_x_y_values['patient1'].unique())
        print('pourcentage *** test_set*************   ',test_x_y_values.shape[0]* 100/all_size )
        print("\n")
        print('pourcentage **** train patients *************   ',train_patient* 100/all_patient )
        print('pourcentage **** test patients *************   ',test_patient* 100/all_patient )


    if(data=='train_set'):

        sub_train_x_y_values=load_pickle( DATA_DIR + '/processed1/XY/sub_train_set_x_y_processed.pickle')

        train_size=x_y_values.shape[0]
        train_patient=len(x_y_values['patient1'].unique())

        sub_train_set_size=sub_train_x_y_values.shape[0]
        sub_train_patient=len(sub_train_x_y_values['patient1'].unique())
        print('pourcentage *** sub_train_set*************   ',sub_train_x_y_values.shape[0]* 100/train_size )

        val_train_x_y_values=load_pickle( DATA_DIR + '/processed1/XY/val_set_x_y_processed.pickle')
        val_set_size=val_train_x_y_values.shape[0]
        val_patient=len(val_train_x_y_values['patient1'].unique())
        print('pourcentage *** val_set *************   ',val_train_x_y_values.shape[0]* 100/train_size )
        print("\n")
        print('pourcentage **** sub_train patients *************   ',sub_train_patient* 100/train_patient )
        print('pourcentage **** val patients *************   ',val_patient* 100/train_patient )

    print("--------------")

    print('number of all patients  '+data +'     ',len(x_y_values['patient1'].unique()))

    is_septic_list=0
    non_septic_list=0

    is_septic=0
    non_septic=0
    group=x_y_values.groupby(['patient1'])
    for i,g in group:
        if np.any(g['label']):
            is_septic +=1
            is_septic_list += g.shape[0]
        else:
            non_septic +=1
            non_septic_list += g.shape[0]

    print ('number of positif patient  ',is_septic)
    print ('number of negatif patient  ',non_septic)
    print('\n')
    print ('number of all observations for positif patients   ',is_septic_list)


    positive_patient_list=x_y_values['patient1'][x_y_values['label']==1.0].unique()
    positive_patient_df=x_y_values[(x_y_values.patient1.isin(positive_patient_list))]
    print('where  \n ',positive_patient_df['label'].value_counts())

    print ('number of all observations for negatif paients   ',non_septic_list)

    #list of negative  patients
    negative_patient_list=x_y_values['patient1'][x_y_values['label']==0.0].unique()
    # construct dataframe of negative patients
    negative_patient_df=x_y_values[x_y_values.patient1.isin(negative_patient_list)]

    #list of positive patients


    print('\n')
    print('number of all observations  '+data +'     ',x_y_values.shape[0])

    print('where  \n',x_y_values['label'].value_counts())
    test_nan(x_y_values,sam=True)
    print("--------------")

#display('mean  '+' display   ',df_mean_st.head())

statistic Model Decision tree or Other

In [ ]:
for data,df_value in [('all_df','all df_values'), ('train_set','train_values'),
                            ('test_set','test_values'),
                           ('sub_train_set','sub_train_values'), ('val_set','val_values') ]:

    fidle.utils.subtitle('-------------' + f'{data}'+'   ----------------')
    #x_y_values=load_pickle(DATA_DIR + '/processed/XY/'+ data + '_x_y_processed.pickle')
    x_y_values=load_pickle(DATA_DIR + '/processed1/XY/'+ data + '_decision_tree_x_y_processed.pickle')

    display(f'{df_value}'+' display  x y ',x_y_values.head())

    #----------------------
    if(data=='all_df'):
        all_size=x_y_values.shape[0]
        all_patient=len(x_y_values['patient1'].unique())
        print('pourcentage **** all df*************   ', '100' +'  %')
        print('pourcentage **** all patient*************   ', '100' +'  %')
        print("\n")
        train_x_y_values=load_pickle( DATA_DIR + '/processed1/XY/train_set_decision_tree_x_y_processed.pickle')

        train_size=train_x_y_values.shape[0]
        train_patient=len(train_x_y_values['patient1'].unique())
        print('pourcentage **** train_set*************   ',train_x_y_values.shape[0]* 100/all_size )

        test_x_y_values=load_pickle( DATA_DIR + '/processed1/XY/test_set_decision_tree_x_y_processed.pickle')

        test_size=test_x_y_values.shape[0]
        test_patient=len(test_x_y_values['patient1'].unique())
        print('pourcentage *** test_set*************   ',test_x_y_values.shape[0]* 100/all_size )
        print("\n")
        print('pourcentage **** train patients *************   ',train_patient* 100/all_patient )
        print('pourcentage **** test patients *************   ',test_patient* 100/all_patient )


    if(data=='train_set'):

        sub_train_x_y_values=load_pickle( DATA_DIR + '/processed1/XY/sub_train_set_decision_tree_x_y_processed.pickle')

        train_size=x_y_values.shape[0]
        train_patient=len(x_y_values['patient1'].unique())

        sub_train_set_size=sub_train_x_y_values.shape[0]
        sub_train_patient=len(sub_train_x_y_values['patient1'].unique())
        print('pourcentage *** sub_train_set*************   ',sub_train_x_y_values.shape[0]* 100/train_size )

        val_train_x_y_values=load_pickle( DATA_DIR + '/processed1/XY/val_set_decision_tree_x_y_processed.pickle')
        val_set_size=val_train_x_y_values.shape[0]
        val_patient=len(val_train_x_y_values['patient1'].unique())
        print('pourcentage *** val_set *************   ',val_train_x_y_values.shape[0]* 100/train_size )
        print("\n")
        print('pourcentage **** sub_train patients *************   ',sub_train_patient* 100/train_patient )
        print('pourcentage **** val patients *************   ',val_patient* 100/train_patient )

    print("--------------")

    print('number of all patients  '+data +'     ',len(x_y_values['patient1'].unique()))

    is_septic_list=0
    non_septic_list=0

    is_septic=0
    non_septic=0
    group=x_y_values.groupby(['patient1'])
    for i,g in group:
        if np.any(g['label']):
            is_septic +=1
            is_septic_list += g.shape[0]
        else:
            non_septic +=1
            non_septic_list += g.shape[0]

    print ('number of positif patient  ',is_septic)
    print ('number of negatif patient  ',non_septic)
    print('\n')
    print ('number of all observations for positif patients   ',is_septic_list)


    positive_patient_list=x_y_values['patient1'][x_y_values['label']==1.0].unique()
    positive_patient_df=x_y_values[(x_y_values.patient1.isin(positive_patient_list))]
    print('where  \n ',positive_patient_df['label'].value_counts())

    print ('number of all observations for negatif paients   ',non_septic_list)

    #list of negative  patients
    negative_patient_list=x_y_values['patient1'][x_y_values['label']==0.0].unique()
    # construct dataframe of negative patients
    negative_patient_df=x_y_values[x_y_values.patient1.isin(negative_patient_list)]

    #list of positive patients


    print('\n')
    print('number of all observations  '+data +'     ',x_y_values.shape[0])

    print('where  \n',x_y_values['label'].value_counts())
    test_nan(x_y_values,sam=True)
    print("--------------")

#display('mean  '+' display   ',df_mean_st.head())